# Reviewing Model Generated Answers

In this notebook we'll load and review answers generated by both the base and fine-tuned models. We only have enough GPU memory to load one of these models at a time so we'll need to clear memory between different model runs!

# Set Base Directories

In [2]:
%env TOKENIZERS_PARALLELISM=false

DATA_DIRECTORY = "../data"
MODEL_DIRECTORY = "../model"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

# Load Train/Test Data Frame

In [3]:
import pandas as pd

path = f"{DATA_DIRECTORY}/train-test-df.csv"
df = pd.read_csv(path, na_filter=False)
print(f"Loaded {df.shape[0]:,d} Train/Test records.")
#print(df.fold.value_counts())
df.sample(n=1)

Loaded 1,050 Train/Test records.


,fold,excerpt,question,answer,hashID
299,7,Birkenstock files for U.S. IPO as listings rec...,What factors have contributed to Birkenstock's...,Birkenstock's decision to file for an IPO in t...,bb48045a4245b64edb69b48849381b59


# Load Evaluation Prompt Prefix

In [4]:
path = f"{DATA_DIRECTORY}/level-1-prompt-prefix.txt"
with open(path) as ifp: prefix = ifp.read()

print(prefix)

Carefully read the excerpt below and then provide a clear concise answer to the follow-up question.


# Load Model, Tokenizer and Generator

In [5]:
import torch
from utilities import display_sample, load_base_model, load_lora

# Clear memory between different model runs!

In [6]:
tokenizer, model, generator = None, None, None
torch.cuda.empty_cache()
!gpustat

ip-172-25-5-124      Thu Oct  5 12:00:18 2023  535.54.03
[0] NVIDIA A10G      | 25'C,   0 % |   309 / 23028 MB |


## Base Model

In [ ]:
tokenizer, model, generator = load_base_model(model_name)

## Fine-tuned Model

In [7]:
fold = 10
directory = f"{MODEL_DIRECTORY}/Llama-2-7b-qa-{fold:02d}"
tokenizer, model, generator = load_lora(MODEL_NAME, directory)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Sample Predictions

In [14]:
x = df.sample(n=1).iloc[0]
display_sample(x, prefix, generator, max_new_tokens=256)

INSTRUCTIONS:
+--------------------------------------------------------------------------------------------------------------------------+
| Carefully read the excerpt below and then provide a clear concise answer to the follow-up question.                      |
+--------------------------------------------------------------------------------------------------------------------------+

EXCERPT:
+--------------------------------------------------------------------------------------------------------------------------+
| Al Jaber noted how "Kenya is closing in on its goal of 100 per cent clean energy by 2030, the African Union's Great      |
| Green Wall is helping reclaim degraded agricultural land across the Sahel. Ethiopia's Green Legacy Initiative is         |
| enhancing food security and stimulating green jobs across the Horn of Africa. And the countries of the Congo Basin are   |
| protecting vital rainforests and helping preserve the world's natural carbon sinks." .  Al Jaber ex

In [13]:
!nvidia-smi


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Oct  5 12:29:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 